In [ ]:
# 21:30 07.05 4 hour work. Пора с этим апи писать работающую тему, показывать Афонину, чтобы он дальше направлял.
!pip install g4f

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.1/600.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.9 MB/s eta 0:00:00


In [ ]:
# 20:00. 5,5 hour work 06.05. Что я планирую сделать?
# 1). В json формат засунуть ответы gpt , пройтись по 5-10 сайтам, возможно со сменой промптов
# проверить что всё работает или найти случаи, когда может не работать и разобраться как это отлавливать.

# 2). дальше скинуть ему скрины(не стесняйся писать!!! я же по факту пишу, а не флужу) и сказать, что вот обработал 5-10 сайтов, вот такие результаты , вроде отличные.
# еще могу сказать ему, что если надо или интересно ему, то могу еще прикрутить тг бота, который по ссылке и промпту вернет всё пользователю
# но приходится долго ждать, поскольку мы дергаем апи и зафайнтюнить тут очев не выйдет
# 3). следующий этап тогда скачать ллм уже локально (лучше на мак, он точно не сломается) , проверить как они будут работать
# если что-то нормальное, то отлично идём файнтюнить (hf Trainer or AutoTrain) читаю доку, там должно быть очень просто по словам Влада и делаю
# 4). в конце концов обсудить с ним(именно с ним, а не с друзьями-кентами, у которых могут быть другие требования)
# что мне нужно писать в самой курсовой, есть ли какой-то жеский формат или я делаю презентацию просто в свободной форме

In [ ]:
test_conferences = [
    "https://www.inf.fu-berlin.de/inst/ag-ti/socg22/socg.html",
    "https://icerm.brown.edu/topical_workshops/tw-24-ips/",
    "https://www.siam.org/conferences/cm/conference/la24",
    "https://www.birs.ca/events/2024/5-day-workshops/24w5313",
    "https://icerm.brown.edu/topical_workshops/tw-24-saso/",
    "https://indico.ictp.it/event/10516",
    "https://icofca.com/",
    "https://moad2024.sciencesconf.org/", # нет спикеров
    "http://caim.romai.ro/Caim24/caim_24.htm", # нет спикеров
    "https://ima.org.uk/23503/4th-ima-conference-on-inverse-problems-from-theory-to-application/", # обрати внимание, чтобы извлекались только спикеры, а не организационный комитет
    "https://sites.google.com/view/atsf2024/home?authuser=0", # нет спикеров, попробуй отдельно отсюда их извлечь https://sites.google.com/view/atsf2024/invited-speakers?authuser=0
    "https://boolean.w.uib.no/bfa-2024/", # опять же проверь, что только нужные спикеры извлекаются, а не орг комитеты еще и тд
    ]

In [3]:
from g4f.client import Client

client = Client()
def ask(promt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": promt}],
    )

    return response.choices[0].message.content

user_agent = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0"
headers = {"User-Agent": user_agent}

In [136]:
import numpy as np
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

def extract_information(conferences):
    texts = {url: "" for url in conferences}
    resps = {url: "" for url in conferences}
    MAXLEN = 20000
    sleep_time = [1, 2, 3, 4, 5]

    for url in tqdm(conferences):
        try:
            request = Request(url, None, headers)
            response = urlopen(request)
        except HTTPError as err:
            texts[url] = None
            resps[url] = "Sorry, I can't follow this link"
            continue

        html = response.read()
        soup = BeautifulSoup(html, features="html.parser")

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        text = text[:MAXLEN]

        texts[url] = text
        promt = 'find and return 4 lists (in structure: (number of list) : [only data in list, without any artifacts]): first includes only: "topics", second  includes only: "speakers", third includes only: "time and date",  fourth includes only:"place" ' + text
        resps[url] = ask(promt)

        sleep(np.random.choice(sleep_time)) # not banned

    return texts, resps

In [137]:
texts, resps = extract_information(test_conferences)

100%|██████████| 12/12 [02:57<00:00, 14.83s/it]


In [172]:
import re

def re_preprocessing(resp):
    resp = resp
    keys = ["topics", "speakers", "time and date", "place"]
    indexes = dict()
    for key in keys:
        s = re.search(r'\d.{0,}' + f'{key}', resp.lower())
        indexes[key] = resp.lower().find(s[0]) if s else -1

    res = dict()
    for i, key in enumerate(keys):
        if i + 1 == len(keys):
            theme = resp[indexes[key]:]
            theme = theme[:theme.find("\n\n")]
        else:
            theme = resp[indexes[key]:indexes[keys[i + 1]]]
        theme = [t.strip() for t in theme.split("\n")[1:] if t]
        theme = [t[1:].strip() if t.startswith("-") else t for t in theme]

        res[key] = theme


    return res

In [183]:
for key, val in re_preprocessing(resps[test_conferences[11]]).items():
    print(key)
    print(val)
    print("\n\n")

topics
['Foundational theory of Boolean functions and discrete structures', 'Design, proposal, and analysis of cryptographically significant (vectorial) Boolean functions', 'The study of applications of (vectorial) Boolean functions to cryptography, codes, and other discrete structures', 'Theory and construction of quantum Boolean functions', 'Theory of finite fields and its applications in cryptography and coding theory']



speakers
['Markus Grassl, University Gdansk', 'Sylvain Guilley, TELECOM-ParisTech', 'Tor Helleseth, University of Bergen, Norway', 'Gregor Leander, Ruhr-Universität Bochum, Germany', 'Gábor Péter Nagy, Budapest University of Technology and Economics, Hungary', 'Claude Carlet, University of Paris VIII, France and University of Bergen, Norway', 'Sihem Mesnager, University of Paris VIII and Sorbonne University North, France', 'Stjepan Picek, Radboud University, the Netherlands']



time and date
['BFA 2024: September 09 – September 13, 2024']



place
['Inter-Univers

In [178]:
print(resps[test_conferences[7]])

1 : [Optimization (Combinatorial, Multi-criteria, Continuous, Dynamic, Stochastic ...), Game Theory, Artificial Intelligence, Statistics, Probability and Stochastic Processes, Control and Dynamical Systems, Graph Theory, Discrete Mathematics, Numbers Theory and Combinatorics, Numerical Analysis and Partial Differential Equations, Mathematical Modeling and Simulation, Applications to Engineering Sciences]
2 : []
3 : [20-22 Oct 2024]
4 : [Tizi ouzou (Algeria)]


In [179]:
print(resps[test_conferences[8]])

1 : [Partial Differential Equations, Ordinary Differential Equations, Dynamical Systems, Mathematical Modeling in Engineering and Life Sciences, Real, Complex, Functional and Numerical Analysis, Probability Theory, Mathematical Statistics, Operations Research, Algebra, Logic, Geometry, Computer Science, Education]
2 : []
3 : [September 19-22, 2024]
4 : [University of Oradea, Universitatii 1, Romania]


In [180]:
print(resps[test_conferences[9]])

1: ["Inverse problems in mathematical and computational imaging.", "Inverse problems in science, medicine, engineering, and other fields.", "Model‐based and data‐driven methods for solving inverse", "Optimisation, statistical, and machine learning methods for solving inverse problems.", "Mathematical theory for inverse problems.", "Deterministic and stochastic computational methods and algorithms."]

2: ["Coralia Cartis (Oxford)", "Marcelo Pereyra (Heriot Watt)", "Olga Hernandez ( Eindhoven University of Technology)", "Rob Scheichl (Heidelberg)"]

3: ["Wednesday September 11, 2024 9:00 am", "Friday September 13, 2024 5:00 pm"]

4: ["3 West North, University of Bath, Claverton Down, Bath , BA2 7AY"]


In [181]:
print(resps[test_conferences[10]])

1: ["Approximation Theory and Special Functions"]
2: ["Prof. George Anastassiou"]
3: ["September 4-7, 2024"]
4: ["TOBB Economics and Technology University, Ankara - Türkiye"]


In [120]:
print(resps[test_conferences[10]])

1. "topics" : ["Approximation Theory", "Special Functions", "Summability Theory", "Applied Mathematics", "Special Functions", "Analysis and Functions Theory"]
2. "speakers" : ["Prof. George Anastassiou"]
3. "time and date" : ["September 4-7, 2024"]
4. "place" : ["Ankara, Türkiye", "TOBB Economics and Technology University"]


In [125]:
"3" in resps[test_conferences[11]]

False

In [ ]:
"""
Результаты тестирования и над чем предстоит работать:

регулярками буду обрабатывать

необработанный случай
1: ['Interacting Particle Systems: Analysis, Control, Learning and Computation'],
2: ['Jose Carrillo','Katy Craig','Massimo Fornasier','Fei Lu','Mauro Maggioni','Kavita Ramanan'],
3: ['Monday, May 6, 2024', 'Tuesday, May 7, 2024'],
4: ['University of Oxford','UC Santa Barbara','Technical University of Munich','Johns Hopkins University','Brown University']

не являются спикерами
2. **Speakers**:
    - Organizing Committee Co-chairs:
        - Laura Grigori, EPFL and PSI, Switzerland
        - Daniel Kressner, EPFL, Switzerland

интересный ответ

2. **Speakers**:
- No speakers listed in the provided text.

нужно отпарсить даты и выбрать самую старшую позднюю, очев она и будет верной как дата конфференции
3: ["Submission deadline: May 15, 2024", "Acceptance notification: July 1, 2024", "Camera-ready submission: August 1, 2024", "Early registration deadline for physical participation: July 15, 2024", "Registration deadline for physical participation: August 17, 2024", "Video submission of presentations: August 20, 2024", "BFA 2024: September 09 – September 13, 2024"],
"""

In [64]:
# в файлы лучше запиши, чтоб нормально видно было мб
for url, resp in list(resps.items()):
    print(url)
    print("\n")
    print(resp)
    print("\n\n")

https://www.inf.fu-berlin.de/inst/ag-ti/socg22/socg.html


Here are the four lists extracted from the provided text:

1. **Topics**:
   - Design, analysis, and implementation of geometric algorithms and data structures;
   - Computational complexity of geometric problems;
   - Implementation and experimental evaluation of geometric algorithms and heuristics, including mathematical, numerical, and algebraic aspects;
   - Discrete and combinatorial geometry;
   - Computational topology, topological data analysis, and topological combinatorics;
   - Applications of computational geometry in any field.

2. **Speakers**:
   - Henry Adams (Colorado State University, USA)
   - SangWon Bae (Kyonggi University, South Korea)
   - Édouard Bonnet (ENS Lyon, France)
   - Timothy Chan (University of Illinois at Urbana-Champaign, USA)
   - Hsien-Chih Chang (Dartmouth College, USA)
   - Hu Ding (University of Science and Technology, China)
   - Vida Dujmović (University of Ottawa, Canada)
   - Herbert

In [ ]:
for url, text in texts.items():
    if text is not None:
        print(url, len(text))
    else:
        print(f"None: {url}")

https://www.inf.fu-berlin.de/inst/ag-ti/socg22/socg.html 15000
https://icerm.brown.edu/topical_workshops/tw-24-ips/ 15000
https://www.siam.org/conferences/cm/conference/la24 11456
https://www.birs.ca/events/2024/5-day-workshops/24w5313 3819
https://icerm.brown.edu/topical_workshops/tw-24-saso/ 11602
https://indico.ictp.it/event/10516 3546
https://icofca.com/ 7187
https://moad2024.sciencesconf.org/ 2484
http://caim.romai.ro/Caim24/caim_24.htm 6235
None: https://ima.org.uk/23503/4th-ima-conference-on-inverse-problems-from-theory-to-application/https://sites.google.com/view/atsf2024/home?authuser=0
https://boolean.w.uib.no/bfa-2024/ 6572


In [148]:
for key, text in texts.items():
    triggers = ["conference", "topic", "speaker", "date", "universit", "committee"]
    if sum(0 if trigger not in text.lower() else 1 for trigger in triggers) == 0:
        print(key)